In [70]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from openpyxl import load_workbook as lw
from openpyxl.utils import get_column_letter

In [71]:
connection_string = """Driver={SQL Server};
            Server=ASVRDB03;
            Database=CONTROLENERGIA;
            UID=daniel.garcia;
            PWD=Daniel2022*;"""

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
conn= con=engine

In [72]:
prueba_historico = pd.read_sql("Select top 100* from historico_consumos", con=engine)

In [73]:
pa_prom_lect_real = pd.read_sql("Select * from pa_prom_lect_real", conn)

In [ ]:
prueba_historico.head(n=5)

In [ ]:
claves = prueba_historico['CLAVE']
claves

In [76]:
prueba_historico.to_excel("prueba_historico.xlsx", index=False)

In [77]:
with pd.ExcelWriter('prueba_historico.xlsx', engine="openpyxl",
                    mode='a') as writer:  
    pa_prom_lect_real.to_excel(writer, sheet_name='Añadido')

In [ ]:
prueba_historico2 = pd.read_sql("Select top 50* from historico_consumos order by clave desc", con=engine)
prueba_historico2

In [79]:
wb = lw('prueba_historico.xlsx')
ws = wb['Sheet1']

In [80]:
len(ws['A'])

101

In [81]:
with pd.ExcelWriter('prueba_historico.xlsx', engine="openpyxl",
                    mode='a', if_sheet_exists='overlay') as writer:  
    prueba_historico2.to_excel(writer, sheet_name='Sheet1', header=None, index=False, startcol=0, startrow=len(ws['A']))

*LINEAS DE ACCIÓN*


# Normalizaciones sin Levantamiento


In [97]:
NORM_SIN_LEV = pd.read_sql('''SELECT a.clave,
       a.os,
       Format(a.fecha_reporte, 'yyMM')                      ciclo_reportado,
       c.fecha_lec_anterior,
       a.fecha_reporte,
       Datediff(day, c.fecha_lec_anterior, a.fecha_reporte) dias_transcurridos,
       f.promedio_diario,
       Round(( Datediff(day, c.fecha_lec_anterior,
             a.fecha_reporte) ) * f.promedio_diario, 0)		csmo_dias_transcurridos,
       f.promedio_mensual,
       a.lectura_acta - Round(( ( Datediff(day, c.fecha_lec_anterior,
       a.fecha_reporte) ) * f.promedio_diario ), 0)         lect_ctrl_ingresar,
       a.fecha_calendario,
       a.fecha_acta,
       Format(a.fecha_acta, 'yyyyMMdd')                     fecha_acta_valid,	--En caso de error validar si el campo fecha_acta IS NULL
       a.dias_para_facturacion								dias_para_facturacion_desde_reporte,
       Datediff(dd, Getdate(), a.fecha_calendario)			dias_para_facturacion_actual,
       c.lectura                                            lectura_ciclo,
       a.lectura_acta,
       a.ult_lect_real,
       a.csmo_dif_lect,
       a.csmo_fact_por_promedios,
       a.csmo_dif_lect - a.csmo_fact_por_promedios          kWh_a_devolver,
       c.lectura - a.lectura_acta                           csmo_dejado_de_fact,
       d.tipo_anomalia,
       e.categoria,
       a.accion,
       b.lect												lect_control_ingresada,
       b.usuario											usuario_ingreso_lect_contrl,
       b.desc_usr,
       b.nom_perfil                                         proceso,
       a.sectoreeh,
       Iif(b.usuario IS NULL, 'No se ingresó lect. ctrl',
       'Si se ingresó lect. ctrl')							estado,
       a.fecha_reg
FROM   pa_norm_sin_lvto a
       LEFT JOIN pa_lecturas_control_st b
              ON Iif(b.nis_rad BETWEEN 3000000 AND 4000000, nis_rad - 3000000,
                 nis_rad)
                 = a.clave
                 AND b.ciclo_actual = Format(a.fecha_reporte, 'yyMM')
       LEFT JOIN historico_consumos c
              ON c.clave = a.clave
                 AND c.ciclo = Format(a.fecha_reporte, 'yyMM')
       LEFT JOIN viewcontrol_resumen d
              ON d.os = a.os
       LEFT JOIN dir_universos_actual e
              ON e.clave = a.clave
                 AND Format(e.fecha_ejecucion_ult_os, 'yyyyMMdd') =
                     Format(a.fecha_acta, 'yyyyMMdd')
       LEFT JOIN (SELECT nis_rad,
                         CASE
                           WHEN nis_rad BETWEEN 3000000 AND 4000000 THEN
                           nis_rad - 3000000
                           ELSE nis_rad
                         END clave,
                         promedio_diario,
                         promedio_mensual
                  FROM   prp_promedios
                  WHERE  periodo = 202211			--COLOCAR CICLO MES ACTUAL
                         AND tip_csmo IN ( 'CO001', 'CO011' )) f
              ON f.clave = a.clave
WHERE  a.accion = 'Ingresar Lect. Control'
       AND ciclo <= Format(Sysdatetime(), 'yyMM')
       --Para ciclos menores o iguales al actual
       AND a.clave NOT IN (SELECT clave
                           FROM   dir_universos_actual
                           WHERE  amerita_cambio_medidor IN ( 1 ))
ORDER  BY a.fecha_reg DESC ''', con=engine)

In [107]:
path_norm_sin_lev = r'C:\Users\daniel.garcia\OneDrive - Empresa Energía Honduras\Cierre\Informes - Seguimiento Cierre\Seguimiento - Norm sin lvto\Seguimiento - Norm sin lvto_v2.xlsx'
wb_NORM = lw(path_norm_sin_lev)
ws_NORM = wb_NORM['Consulta1']
len(ws_NORM['A'])


202

In [108]:
with pd.ExcelWriter(path_norm_sin_lev, engine="openpyxl",
                    mode='a', if_sheet_exists='new') as writer:  
    NORM_SIN_LEV.to_excel(writer, sheet_name='Consulta2', index=False,)

Nueva linea